In [1]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np

In [3]:
column_of_interest = ["text_ tokens", "engaging_user_id", "reply_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/train_set_reply.csv', encoding="utf-8",
                     usecols= [1, 4, 5])

In [26]:
len(train_set)

133267714

In [24]:
train_set.head()

,text_ tokens,engaging_user_id,reply_timestamp
0,101\t108\t31278\t90939\t70325\t196\t199\t71436...,12A1AF0088C5B4FCBFA024D4A1971323,NaN
1,101\t95026\t90141\t54922\t10287\t13567\t10169\...,C836F432B0FC7847C2C753949C51B961,NaN
2,101\t10117\t12518\t45415\t10472\t10114\t19072\...,77309DE08AB631D8224254D33648F287,NaN
3,101\t36084\t11372\t26095\t117\t12900\t39122\t1...,91363F238C79DAEE42E2A2C97A5F8E7C,NaN
4,101\t34764\t26905\t117\t10479\t92486\t10559\t9...,EFA4C5B62E097EB203F8AFDC470AEB27,NaN


In [5]:
column_of_interest = ["text_ tokens", "engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val_set_reply.csv', encoding="utf-8",
                     usecols= [1, 4])

In [6]:
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()

In [25]:
train_set_positive = train_set[train_set.reply_timestamp.notnull()]

In [27]:
train_set_positive.head()

,text_ tokens,engaging_user_id,reply_timestamp
39,101\t32310\t10269\t10271\t26506\t10112\t106\t1...,E0F901FA8F5D1A7B777FD1871A913910,1.580948e+09
42,101\t100\t9665\t12178\t9415\t14523\t9357\t1073...,CE5D706649EEA7D04F2D9BFFACEBF02E,1.580970e+09
161,101\t35936\t48628\t117\t146\t100\t181\t10978\t...,95EADB3CF75EAB9ECF99DE96328B91C8,1.580947e+09
311,101\t10377\t10124\t30767\t144\t53668\t12360\t1...,D3AB4517B014973FA8224E3440C68771,1.581046e+09
525,101\t20520\t484\t28280\t469\t12526\t46351\t466...,66CCBBE5099558476C0EEB226BBA4474,1.580947e+09


In [28]:
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')

In [29]:
warm_train_set_positive.to_csv('s3://recsys-challenge-2020/warm_train_set_positive.csv', index = False)

In [ ]:
warm_train_set = pd.read_csv('s3://recsys-challenge-2020/warm_train_set.csv', encoding="utf-8")

In [30]:
warm_train_set_positive.head()

,text_ tokens,engaging_user_id,reply_timestamp
0,101\t35936\t48628\t117\t146\t100\t181\t10978\t...,95EADB3CF75EAB9ECF99DE96328B91C8,1.580947e+09
1,101\t14516\t56920\t11593\t10124\t61758\t10105\...,95EADB3CF75EAB9ECF99DE96328B91C8,1.580991e+09
2,101\t84199\t10142\t11426\t98507\t10165\t11084\...,95EADB3CF75EAB9ECF99DE96328B91C8,1.581006e+09
3,101\t69841\t10271\t27904\t12277\t13028\t21964\...,95EADB3CF75EAB9ECF99DE96328B91C8,1.581044e+09
4,101\t107990\t96666\t10376\t34462\t131\t119\t12...,95EADB3CF75EAB9ECF99DE96328B91C8,1.581138e+09


In [10]:
len(warm_train_set)

63067118

In [31]:
len(warm_train_set_positive)

1680295

In [11]:
warm_train_set_positive = warm_train_set[warm_train_set.reply_timestampstamp.notnull()]

In [12]:
len(warm_train_set_positive)

1680295

In [14]:
warm_train_set_positive.head()

,text_ tokens,engaging_user_id,reply_timestamp
31,101\t10224\t11617\t101480\t10133\t18958\t10133...,DEFC3A3F681051E5B56AB7F1AEF273E1,1.581267e+09
55,101\t137\t25923\t15478\t168\t34218\t10160\t162...,A6ECB6ABA45EB75E7F8D5C65C5F151A7,1.581020e+09
66,101\t30929\t24558\t146\t41549\t15127\t43310\t1...,0A8073DE7855DC2154FF2DFBA793FC4C,1.581491e+09
139,101\t12034\t10189\t15263\t15688\t11044\t56157\...,850E3EE1A01FE2D52B064B05A30805B1,1.581455e+09
256,101\t32849\t19847\t24283\t10129\t10212\t12282\...,EEACFA2E013FFC312D5980CD8A9397BB,1.581362e+09


In [32]:
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()

In [33]:
user_tokens.head()

,engaging_user_id,text_ tokens
0,00000B85AAF7DE172876FD96718C4469,[101\t12882\t10105\t24699\t10124\t12820\t169\t...
1,00000E0C9B364891CDE89ECFC54771DE,[101\t56898\t137\t60818\t20712\t10133\t37115\t...
2,0000142A2D3FCA2A1EAE0569CBEA8A11,[101\t56898\t137\t44798\t10858\t16896\t11083\t...
3,0000170273D2530A0DF580401CC32AE0,[101\t13999\t12845\t34396\t11088\t196\t10838\t...
4,00003C08A3D1CEF5F68EA6F17234F972,[101\t11589\t131\t146\t11337\t12253\t124\t1917...


In [22]:
len(user_tokens)

1182489

In [34]:
len(user_tokens)

1182489

In [36]:
len(unique_val_set_users)

10173370

In [35]:
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens.csv', index = False)

In [ ]:
len(train_set)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [ ]:
iterator = 0
for chunk in np.array_split(warm_train_set, 10000):
    print(iterator)
    iterator = iterator + 1
    df_embeddings = pd.DataFrame()
    df_embeddings = chunk[["engaging_user_id"]]
    df_embeddings['text_embeddings'] = chunk['text_ tokens'].\
apply(lambda x : model(torch.tensor(list(map(int, x.split('\t')))).unsqueeze(0))[0][0][0])
    df_embeddings.to_csv('s3://recsys-challenge-2020/embeddings_user.csv', mode='a', header=False)

In [ ]:
iterator = 0
for chunk in np.array_split(train_set, 10000):
    print(iterator)
    iterator = iterator + 1
    df_embeddings = pd.DataFrame()
    df_embeddings['text_embeddings'] = chunk['text_ tokens'].\
apply(lambda x : model(torch.tensor(list(map(int, x.split('\t')))).unsqueeze(0))[0][0][0])
    df_embeddings.to_csv('s3://recsys-challenge-2020/embeddings.csv', mode='a', header=False)

In [ ]:
column_of_interest = ["text_ tokens"]
train_set = pd.read_csv('s3://recsys-challenge-2020/train_set.csv', encoding="utf-8",
                     usecols= [1])